this notebooks looks at differences in ebike trips and non-ebike trips in terms of trip duration, trip distance, and distance/duration using the responses from the mapbox api

In [121]:
import pandas as pd

In [122]:
#import trips with ebike identifier and directions csv
trips = pd.read_csv('trips_ebike_id.csv')
directions = pd.read_csv('directions.csv')

In [123]:
#filter one timeframe starting at time of ebike introduction
trips = trips[trips['starttime']>='2018-08-20']

In [62]:
trips.head()

,Unnamed: 0,Unnamed: 0.1,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,value_x,value_y,type
10700351,10700351,1162590,757,2018-08-20 00:00:09.467,2018-08-20 00:12:47.373,386.0,Centre St & Worth St,40.714948,-74.002345,394.0,E 9 St & Avenue C,40.725213,-73.977688,30808,Subscriber,1984,1,0.0,0.0,0.0
10700352,10700352,1162591,358,2018-08-20 00:00:14.638,2018-08-20 00:06:13.576,2023.0,E 55 St & Lexington Ave,40.759681,-73.970314,3136.0,5 Ave & E 63 St,40.766368,-73.971518,21382,Subscriber,1994,1,0.0,0.0,0.0
10700353,10700353,1162592,427,2018-08-20 00:00:17.737,2018-08-20 00:07:24.765,3314.0,W 95 St & Broadway,40.793770,-73.971888,3357.0,W 106 St & Amsterdam Ave,40.800836,-73.966449,20847,Subscriber,1999,2,0.0,0.0,0.0
10700354,10700354,1162593,1545,2018-08-20 00:00:22.676,2018-08-20 00:26:08.182,281.0,Grand Army Plaza & Central Park S,40.764397,-73.973715,3163.0,Central Park West & W 68 St,40.773407,-73.977825,28424,Customer,1969,0,0.0,0.0,0.0
10700355,10700355,1162594,731,2018-08-20 00:00:23.313,2018-08-20 00:12:34.777,3121.0,Jackson Ave & 46 Rd,40.745248,-73.947333,3116.0,Huron St & Franklin St,40.732660,-73.958260,33372,Customer,1969,0,0.0,0.0,0.0


In [139]:
#create unique id 'concat' between both dataframes and join them on 'concat'
directions['source'] = directions['source'].astype(str) 
directions['target'] = directions['target'].astype(str) 
directions['concat'] = directions['source'] + '-' + directions['target']

trips['start station id'] = trips['start station id'].astype(int).astype(str)
trips['end station id'] = trips['end station id'].astype(int).astype(str)
trips['concat'] = trips['start station id'] + '-' + trips['end station id']

trips_directions = trips.merge(directions, on='concat', how='left')

In [140]:
#create new dataframe only with the columns we need
trips_directions = trips_directions[['tripduration','source','target','geometry','type_x','type_y','count','distance','duration']]

In [141]:
#filter out null values
trips_directions = trips_directions[~trips_directions['geometry'].isnull()]

In [142]:
#rename some columns
trips_directions = trips_directions.rename(columns={'geometry':'coords','type_x':'is_ebike','type_y':'line'})
trips_directions['geometry'] = trips_directions['line'] + trips_directions['coords']

In [143]:
#change distance from meters to miles, and duration from seconds to minutes
trips_directions['dist_durat'] = trips_directions['distance']/trips_directions['duration']

In [144]:
trips_directions.groupby('is_ebike')['distance','tripduration','dist_durat'].mean()

,distance,tripduration,dist_durat
is_ebike,,,
0.0,2096.548328,762.728991,3.568102
1.0,2104.313039,868.176800,3.564633
